# Homework



In [0]:
# Update torch, torchvision and numpy
!pip install -U torch torchvision numpy opencv-python

In [0]:
# Install homework repository
!git clone https://github.com/szykry/CVML_HW.git

# Set repo as root folder
import os
name = "/content/CVML_HW/"
os.chdir(name)
!pwd

In [0]:
# Homework dataset
!wget http://deeplearning.iit.bme.hu/CVS/HW.zip
!unzip -qq HW.zip
!rm HW.zip

# Traffic Sign Classification set
!wget http://deeplearning.iit.bme.hu/CVS/trafficSignsHW.zip
!unzip -qq trafficSignsHW.zip
!rm trafficSignsHW.zip

In [0]:
!unzip -qq pics.zip
!rm pics.zip

In [0]:
import cv2
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

for dirpath, dirnames, filenames in os.walk("./pics"):
  for filename in filenames:
    print(filename)
    a=cv2.imread("/content/CVML_HW/pics/"+filename)
    a=cv2.cvtColor(a,cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.imshow(a)

# Tradition



In [0]:
#OpenCV
import cv2

#Numpy - numeric library
import numpy as np

#Plotting
import matplotlib.pyplot as plt

#This way it doesn't try to open a window un the GUI - works in python notebook
%matplotlib inline

def minMaxLoc(img):
  Xmin = 999;
  Xmax = 0;
  Ymin = 999;
  Ymax = 0;
  
  for i in range(img.shape[0]):
    sor =img[i,:]
    mean=np.count_nonzero(sor)
    if ((Ymin == 999) & (mean > 0)):
      Ymin=i
    if ( mean > 0):
      Ymax = i

  for i in range(img.shape[1]):
    oszlop =img[:,i]
    mean=np.count_nonzero(oszlop)
    if ((Xmin == 999) & (mean > 0)):
      Xmin=i
    if ( mean > 0):
      Xmax = i

  return Xmin,Xmax,Ymin,Ymax

def watershed(edges,img,debug=False):
  kernel = np.ones((3,3),np.uint8)
  #widen edges to make them touch
  edges = cv2.dilate(edges, kernel,iterations=1)

  ret, thresh = cv2.threshold(edges,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
  if debug:
    plt.figure(figsize=(7,7))
    plt.imshow(thresh,cmap='gray')

  sure_fg = cv2.erode(thresh,kernel,iterations=1)
  sure_bg = cv2.dilate(thresh,kernel,iterations=1)

  # Uncertain region(borders)
  unknown = cv2.subtract(sure_bg,sure_fg)

  if debug:
    plt.figure(figsize=(7,7))
    plt.imshow(sure_fg,cmap='gray')

  # Label separate foreground pathes
  ret, markers = cv2.connectedComponents(sure_fg)
      
  # Add one to all labels so that sure background is not 0, but 1
  markers = markers+1
  # Now, mark the region of unknown with zero
  markers[unknown==255] = 0

  if debug:
    plt.figure(figsize=(7,7))
    plt.imshow(markers)
    
  # Runwatershed
  markers = cv2.watershed(img,markers)
  if debug:
    plt.figure(figsize=(15,15))
    plt.subplot(1,2,1)
    plt.imshow(markers)
    plt.subplot(1,2,2)
    plt.imshow(img)

  return markers

def depth_segmenter(debug=False):
  # Read images
  img = cv2.imread("../HW/g3/rgb/250.jpg")
  depth = cv2.imread("../HW/g3/depth/250.png", -1)
  depth2 = cv2.imread("../HW/g3/depth/250.png")

  sor =depth[0,:]
  legelsoAtlag=np.true_divide(sor.sum(),np.count_nonzero(sor))

  #hatsoMaszk=cv2.inRange(depth, legelsoAtlag*0.9, np.amax(depth))
  hatsoMaszk =  np.zeros((480,640)).astype("uint8")
  hatsoMaszk[depth<legelsoAtlag*0.7]=[1]

  if debug:
    plt.figure(figsize=(20,20))
    plt.imshow(hatsoMaszk,cmap='gray')

  # convert to RGB
  img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img_hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

  lower_range = np.array([90,0,0])
  upper_range = np.array([120,255,220])

  maszk=cv2.inRange(img_hsv, lower_range, upper_range)

  maszkNegative = cv2.bitwise_not(maszk)
  
  if debug:
    out = cv2.bitwise_and(img_hsv,img_hsv,mask = maszkNegative)
    plotolni = cv2.cvtColor(out, cv2.COLOR_HSV2RGB)
    plt.figure(figsize=(20,20))
    plt.imshow(plotolni,cmap='gray')

  hue_channel = img_hsv[:,:,0]
  avg_hue = np.average(hue_channel)
  asdasd, hue_channel_segment = cv2.threshold(hue_channel,avg_hue,255,cv2.THRESH_BINARY)
  kernel = np.ones((5,5),np.uint8)
  maszkDilation = cv2.erode(hue_channel_segment,kernel,iterations = 2)

  maszkDilation = cv2.bitwise_not(maszkDilation)
  maszkNegative = cv2.bitwise_and(maszkDilation,maszkDilation,mask = maszkNegative)
  maszkNegative = cv2.bitwise_and(maszkNegative,maszkNegative,mask = hatsoMaszk)
  
  if debug:
    plt.figure(figsize=(20,20))
    plt.imshow(maszkNegative,cmap='gray')

  # calculate foreground
  for i in range(depth.shape[0]):
    sor =depth[i,:]
    mean=np.true_divide(sor.sum(),np.count_nonzero(sor))
    sor[sor>mean-10]=0

  # for some reason doesn't work without this
  depth2[depth==0]=0
  img_rgb = np.where(depth2 != 0, img_rgb, [0,0,0])

  # Figure with subplots
  if debug:
    plt.figure(figsize=(30,30))
    plt.subplot(1,2,1)
    plt.imshow(img_rgb) 
    plt.subplot(1,2,2)
    plt.imshow(depth,cmap='gray')

  # convert to uint8 from uint16
  cv2.normalize(depth, depth, 0, 255, cv2.NORM_MINMAX)
  depth=depth.astype('uint8')
  depth_8 = cv2.cvtColor(depth, cv2.COLOR_GRAY2BGR)

  # get edges
  edges = cv2.Canny(depth_8,50,120)
  markers = watershed(edges,img,debug=True)

  ####
  img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  for i in np.unique(markers):
    if i<2 :
      continue

    imgcopy=img.copy()
    imgcopy[markers!=i]=[0,0,0]  
    xmin,xmax,ymin,ymax = minMaxLoc(imgcopy)

    if ((xmax-xmin > 500) & (ymax-ymin > 350)):
      continue
    
    imgcopy = cv2.bitwise_and(imgcopy,imgcopy,mask = maszkNegative)

    if np.count_nonzero(imgcopy)<3600:
      continue

    #if np.var(np.nonzero(cv2.cvtColor(imgcopy,cv2.COLOR_RGB2HSV)[:,:,0]))<4200:
    #  continue
    if debug:
      plt.figure(figsize=(10,10))
      plt.imshow(imgcopy)
      print(str(i)+":"+str(np.var(np.nonzero(cv2.cvtColor(imgcopy,cv2.COLOR_RGB2HSV)[:,:,0])))+"........\t"+str(np.count_nonzero(imgcopy))+"minmax:"+str(xmin)+" "+str(xmax)+" "+str(ymin)+" "+str(ymax))

  if debug:
    plt.subplot(1,3,3)
    plt.imshow(cv2.imread("../HW/g1/depth/1.png", -1),cmap='gray')

depth_segmenter(debug=True)

In [0]:
# Display the first images
colors = [(0,0,255),(255,0,255),(0,255,0)]

def drawBBs(BBs, img):
    img = cv2.resize(img, (1280, 960))
    for BB in BBs:
        u = BB[0]*2
        v = BB[1]*2
        w = BB[2]*2
        h = BB[3]*2
        c = BB[4]
        sc = BB[5]
        x = BB[6]
        y = BB[7]
        z = BB[8]
        s = (u - w // 2, v - h // 2)
        e = (u + w // 2, v + h // 2)
        cv2.rectangle(img, s, e, colors[c], 1)
        tl = (s[0], s[1]+15)
        bl = (s[0], e[1]-5)
        cv2.putText(img,subclassNames[c][sc],tl,cv2.FONT_HERSHEY_COMPLEX_SMALL,0.75,colors[c])
        coords = "(%.2f, %.2f, %.2f)" % (x,y,z)
        cv2.putText(img,coords,bl,cv2.FONT_HERSHEY_COMPLEX_SMALL,0.65,colors[c])
    
    return img

In [0]:
# Get all subfolders in a directory
import os
myFolderList = [f.path for f in os.scandir(path) if f.is_dir()]

In [0]:
# Get all files with extension in a directory
import glob
import re
 
def sorted_nicely( l ):
    """ Sort the given iterable in the way that humans expect."""
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(l, key = alphanum_key)
 
names = sorted_nicely(glob.glob1(path, "*.extension"))

In [0]:
# Class names
classNames = ['traffic sign', 'vehicle', 'cactus']
subclassNames = [
    ['Bump', 'Bumpy road', 'Bus stop', 'Children', 'Crossing (blue)', 'Crossing (red)', 'Cyclists',
     'Danger (other)', 'Dangerous left turn', 'Dangerous right turn', 'Give way', 'Go ahead', 'Go ahead or left',
     'Go ahead or right', 'Go around either way', 'Go around left', 'Go around right', 'Intersection', 'Limit 100',
     'Limit 120', 'Limit 20', 'Limit 30', 'Limit 50', 'Limit 60', 'Limit 70', 'Limit 80', 'Limit 80 over',
     'Limit over', 'Main road', 'Main road over', 'Multiple dangerous turns', 'Narrow road (left)',
     'Narrow road (right)', 'No entry', 'No entry (both directions)', 'No entry (truck)', 'No stopping', 'No takeover',
     'No takeover (truck)', 'No takeover (truck) end', 'No takeover end', 'No waiting', 'One way road',
     'Parking', 'Road works', 'Roundabout', 'Slippery road', 'Stop', 'Traffic light', 'Train crossing',
     'Train crossing (no barrier)', 'Wild animals', 'X - Priority', 'X - Turn left', 'X - Turn right'],
    ['SUV','truck','plane'],
    ['happy','sad','angry','evil']
]

# Deep Learning

In [0]:
# Use --help to list the available arguments
# !python train.py

In [0]:
import torch
import torch.nn as nn
import torchvision
import os
import sys
import glob
import re
import numpy as np
import matplotlib.pyplot as plt

from torch import optim
from torchvision import transforms, datasets, get_image_backend
from torchvision.datasets import ImageFolder
from PIL import Image
from IPython.display import HTML, display

%matplotlib inline


haveCuda = torch.cuda.is_available()
numEpoch = 50
batch_size = 64     # train set: 104027, test set: 10418 -> useful batch sizes: 32 or 64
num_workers = 4
train_dir = './trafficSignsHW/trainFULL'
test_dir = './trafficSignsHW/testFULL'
targets = ['Bump', 'Bumpy road', 'Bus stop', 'Children', 'Crossing (blue)', 'Crossing (red)', 'Cyclists',
           'Danger (other)', 'Dangerous left turn', 'Dangerous right turn', 'Give way', 'Go ahead', 'Go ahead or left',
           'Go ahead or right', 'Go around either way', 'Go around left', 'Go around right', 'Intersection', 'Limit 100',
           'Limit 120', 'Limit 20', 'Limit 30', 'Limit 50', 'Limit 60', 'Limit 70', 'Limit 80', 'Limit 80 over',
           'Limit over', 'Main road', 'Main road over', 'Multiple dangerous turns', 'Narrow road (left)',
           'Narrow road (right)', 'No entry', 'No entry (both directions)', 'No entry (truck)', 'No stopping', 'No takeover',
           'No takeover (truck)', 'No takeover (truck) end', 'No takeover end', 'No waiting', 'One way road',
           'Parking', 'Priority', 'Road works', 'Roundabout', 'Slippery road', 'Stop', 'Traffic light', 'Train crossing',
           'Train crossing (no barrier)', 'Turn left', 'Turn right', 'Wild animals'
           ]


def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))


def plotResults():
    # X coordinate for plotting
    x = np.arange(numEpoch)

    plt.figure(figsize=(20,10))

    # Train is red, validation is blue
    plt.subplot(1,2,1)
    plt.plot(x,train_accs,'r')
    plt.plot(x,val_accs,'b')

    plt.subplot(1,2,2)
    plt.plot(x,train_losses,'r')
    plt.plot(x,val_losses,'b')

    plt.show()


def transformData():

    transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),   # random shift
        transforms.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=3, fill=0),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    return transform, transform_val


class Conv(nn.Module):
  
    def __init__(self, in_channels, channels, k_size=3, stride=1):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channels, channels, k_size, stride=stride, padding=k_size//2, bias=False)
        self.bn = nn.BatchNorm2d(channels)
        
    def forward(self,x):
        return self.bn(torch.relu(self.conv(x)))


class ConvNet(nn.Module):
    
    def __init__(self, base_channels=16, in_channels=3, num_classes=55):  # base: csatorna növekedéshez, in: RGB
        super().__init__()
        
        # Filters
        self.c11 = Conv(in_channels, base_channels)
        self.c12 = Conv(base_channels, base_channels)
        self.d1  = Conv(base_channels, base_channels*2, stride=2) # Downscale using strided convolution and expand channels
        
        self.c21 = Conv(base_channels*2, base_channels*2)
        self.c22 = Conv(base_channels*2, base_channels*2)
        self.d2  = Conv(base_channels*2, base_channels*4, stride=2)
        
        self.c31 = Conv(base_channels*4, base_channels*4)
        self.c32 = Conv(base_channels*4, base_channels*4)
        self.d3  = Conv(base_channels*4, base_channels*8, stride=2)
        
        self.c41 = Conv(base_channels*8, base_channels*8)
        self.c42 = Conv(base_channels*8, base_channels*8)
        self.d4  = Conv(base_channels*8, base_channels*16, stride=2)
        
        self.c51 = Conv(base_channels*16, base_channels*16)
        self.c52 = Conv(base_channels*16, base_channels*16)
        self.d5  = Conv(base_channels*16, base_channels*32, stride=2) 

        # Input image is 32x32 -> after 5 downscaling the activation map is 1x1
        # [batch, ch, h, w]  -> [batch, ch] linearisba, h=1, w=1
        # Classifier is a normal 1x1 convolution that produces num_classes class scores
        # This layer does not have BatchNorm or ReLU

        self.classifier = nn.Conv2d(base_channels*32, num_classes, kernel_size=1)
        
    def forward(self,x):

        x = self.d1(self.c12(self.c11(x)))
        x = self.d2(self.c22(self.c21(x)))
        x = self.d3(self.c32(self.c31(x)))
        x = self.d4(self.c42(self.c41(x)))
        x = self.d5(self.c52(self.c51(x)))
        
        return torch.squeeze(self.classifier(x))    # After squeeze is becomes (batch_size x num_classes)


def train(epoch, trainLoader):

    running_loss = 0.0
    correct = 0.0
    total = 0

    net.train()

    bar = display(progress(0, len(trainLoader)), display_id=True)     # Create progress bar

    for i, data in enumerate(trainLoader, 0):

        inputs, labels = data
        if haveCuda:
            inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        with torch.no_grad():

            running_loss += loss.item()     # Accumulate loss
            _, predicted = torch.max(outputs, 1)  # [batch, num_class] -> [batch, 1]
            correct += predicted.eq(labels).sum().item() # Count how many of the predictions equal the labels
            total += labels.shape[0]   # Accumulate number of total images seen

        bar.update(progress(i+1, len(trainLoader)))

    tr_loss = running_loss / i
    tr_corr = correct / total * 100
    print("Train epoch %d loss: %.3f correct: %.2f" % (epoch + 1, running_loss / i, tr_corr))

    return tr_loss,tr_corr


def val(epoch,testLoader):

    running_loss = 0.0
    correct = 0.0
    total = 0

    net.eval()

    bar = display(progress(0, len(testLoader)), display_id=True)

    for i, data in enumerate(testLoader, 0):

        inputs, labels = data
        if haveCuda:
            inputs, labels = inputs.cuda(), labels.cuda()

        with torch.no_grad():

            outputs = net(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += predicted.eq(labels).sum().item()
            total += labels.shape[0]

        bar.update(progress(i+1, len(testLoader)))

    val_loss = running_loss / i
    val_corr = correct / total * 100
    print("Test epoch %d loss: %.3f correct: %.2f" % (epoch + 1, running_loss / i, val_corr))

    return val_loss,val_corr


def testModel():

    inputs, labels = next(iter(train_loader))
    if haveCuda:
        inputs, labels = inputs.cuda(), labels.cuda()

    net = torch.load("./model/pyVision_full_tf.pth")
    net.eval()
    outputs = net(inputs)

    _, predicted = torch.max(outputs, 1)

    mean = torch.Tensor((0.485, 0.456, 0.406)).unsqueeze(1).unsqueeze(1)
    std = torch.Tensor((0.229, 0.224, 0.225)).unsqueeze(1).unsqueeze(1)


    f, axarr = plt.subplots(batch_size//8, 8,figsize=(30, 20)) # 64 images

    for i,(img,pred) in enumerate(zip(inputs,predicted)):

        img_rescaled = img.cpu() * std + mean         # undo the normalization
        
        name = targets[pred.cpu().item()]
        
        axarr[i//8,i%8].imshow(img_rescaled.permute(1,2,0))         # Permute dimensions
        
        axarr[i//8,i%8].set_title(name)
        
        axarr[i//8,i%8].grid(True)
        
        axarr[i//8,i%8].set_xticks([])
        axarr[i//8,i%8].set_yticks([])


if __name__ == '__main__':

    transform, transform_val = transformData()
  
    train_dataset = datasets.ImageFolder(train_dir, transform)
    test_dataset = datasets.ImageFolder(test_dir, transform_val)
                          
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               shuffle=True,
                                               batch_size=batch_size,
                                               num_workers=num_workers
                                               )
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=batch_size,
                                              num_workers=num_workers
                                              )

    train_accs = []
    train_losses = []
    val_accs = []
    val_losses = []

    best_acc = 0

    tfLearning = True

    if not tfLearning:
        torch.manual_seed(1)  # Set pseudo-random generator seeds to make multiple runs comparable
        if haveCuda:
            torch.cuda.manual_seed(1)

        net = ConvNet()
        
    else:
        net = torch.load("./model/pyVision_full_tf.pth")

    if haveCuda:
        net = net.cuda()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=1e-1, momentum=0.9, nesterov=True, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,numEpoch,eta_min=1e-2)

    for epoch in range(numEpoch):
        
        loss,acc = train(epoch,train_loader)
        train_accs.append(acc)
        train_losses.append(loss)

        loss,acc = val(epoch,test_loader)
        val_accs.append(acc)
        val_losses.append(loss)

        scheduler.step()

        if not tfLearning:
          if acc > best_acc:
              print("Best Model, Saving")
              best_acc = acc
              torch.save(net,"./model/pyVision_full_tf.pth")

        else:
          if acc > 99.5:
              print("Best Model, Saving")
              best_acc = acc
              torch.save(net,"./model/pyVision_full_tf.pth")
    
    # Results
    plotResults()

    testModel()

Train epoch 1 loss: 0.193 correct: 94.69


Test epoch 1 loss: 0.075 correct: 97.80


Train epoch 2 loss: 0.100 correct: 97.23


Test epoch 2 loss: 0.058 correct: 98.41


Train epoch 3 loss: 0.090 correct: 97.53


Test epoch 3 loss: 0.031 correct: 99.23


Train epoch 4 loss: 0.091 correct: 97.51


Test epoch 4 loss: 0.063 correct: 98.15


In [0]:
import os
import glob
import re

def sorted_nicely( l ):
    """ Sort the given iterable in the way that humans expect."""
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(l, key = alphanum_key)


myFolderList = [f.path for f in os.scandir('./trafficSignsHW/trainFULL') if f.is_dir()]
myFolderList.sort()

for folder in myFolderList:
  img_paths = sorted_nicely(glob.glob1(folder, "*.jpg"))

In [0]:
# Model's parameters
net = torch.load("./model/pyVision_full_s.pth")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

In [0]:
import cv2

img = cv2.imread("./trafficSignsHW/testFULL/Priority/g1_89_3.1ts.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(5,5))
plt.imshow(img)

img = torch.from_numpy(img)
img = img.permute(2,0,1)

input = torch.zeros(1,3,32,32)
input[0] = img

if haveCuda:
    input = input.cuda()

net = torch.load("./model/pyVision_full_tf.pth")
net.eval()
output = net(input)

_, pred = torch.max(output, 0)

print(targets[pred.cpu().item()])

# Evaluation

In [0]:
# (u, v) are the center coordinates of the object's bounding box, while (w, h) are the width and height parameters. All four are expected in pixels @640x480 resolution!
# (x, y, z) are the 3D coordinates of the object relative to the camera. They are expected in meters!

myObjects = [u, v, w, h, classInd, subClassInd, x, y, z]

In [0]:
# poses = np.zeros((12,), dtype=int)  => 3D HC

myPred = {
    'poses' : [t_11, t12_, t_13, t_14, ..., t_33, t_34],
    'objects' : [obj_1, obj_2, ... obj_n]
}

In [0]:
myAnswers = {
    'HW/g1/rgb/1.jpg' : myPred1,
    'HW/g1/rgb/123.jpg' : myPred2,
    ...
    'HW/g4/rgb/95.jpg' : myPredX,
}

In [0]:
from HW.evaluate import evaluate

file = open('all_predictions.pickle','rb')
predictions = pickle.load(file)

evaluate(predictions)